In [1]:
from emgdecompy.decomposition import *
from emgdecompy.contrast import *
from emgdecompy.viz import *
from emgdecompy.preprocessing import *
from db1_preprocess_utils import *
from feature_extraction import *
from experiment_one_utils import *

2023-09-04 17:33:37.372839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-04 17:33:40.819615: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-04 17:33:48.497569: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bmis/anaconda3/envs/tf/lib/python3.9/site-packages/nvidia/cudnn/lib:/home/bmis/anaconda3/envs/tf/lib/:
2023-09-04 17:33:48.531510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [2]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold
from scipy.signal import stft

In [3]:
path = '/home/bmis/Documents/AI-Workspace/sEMG/GestureClassificationUsingCViT/data/DB1/raw'

signal_type='raw'
input_type='raw'
n_channels=64
low_cut=20
high_cut=50
order=6 
window_size=50
overlap=50
fs=2048

whiten=True
center=False
extend=False
extend_size=0
normalize=False
mu=0

start_subject = 1
total_subject = 20
session=[1,2]
ratio = 0.2

noise_db = [5,10,15]
std = 1
stochastic_depth_rate = 0.1

In [4]:
data, label, X_test, y_test = get_experiment_data(path, subjects=start_subject, sessions=session,
                                                       signal_type='raw', input_type=input_type, 
                                                       channels=n_channels, low_cut=20, high_cut=500,
                                                      order=6, window_size=window_size, overlap=overlap, fs=fs, 
                                                      extend=extend, center=center, 
                                                      extend_size=extend_size, whiten=whiten,
                                                      normalize=False, mu=0, ratio=0.2)

####################################################################################
Loading subject 1
####################################################################################
Whiten.
Size of the input data is (17152, 64, 102, 1)
The input label shape is (17152,)
The total number of classes is 34
************************************************************************************
Loaded RAW input data
************************************************************************************


In [5]:
data.shape, label.shape, X_test.shape, y_test.shape

((17152, 64, 102, 1), (17152,), (4288, 64, 102), (4288,))

In [6]:
#Hyper-parameter

input_shape = data.shape[1:]
patch_size = 8
n_patches = 24 
dims = n_patches + 1 
n_transformer_layers = 2
n_heads = 2
transformer_units = [dims * 2,dims]  # Size of the transformer layers
mlp_head_units = [128, 34]   # [2048, 1024]
n_classes = len(np.unique(label))
stochastic_depth = False

stochastic_depth_rate = 0.1

n_batches = 32
n_epochs = 150
n_folds = 3
type_of_experiment = 2

In [7]:
model = vit_gesture_classification(input_shape, dims, patch_size, n_patches,
                                  n_transformer_layers, n_heads, transformer_units,
                                   mlp_head_units, n_classes,stochastic_depth,stochastic_depth_rate)

model.summary()

2023-09-04 17:34:23.016240: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-04 17:34:25.066533: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-04 17:34:25.066778: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-04 17:34:25.092881: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "sEMG-Decomposition"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 102, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 covolution_layer (Sequential)  (None, 32, 51, 25)   20413       ['input_1[0][0]']                
                                                                                                  
 generate__patches (Generate_Pa  (None, None, 1600)  0           ['covolution_layer[0][0]']       
 tches)                                                                                           
                                                                                 

In [8]:
run_experiment(n_folds, n_batches, n_epochs, start_subject, total_subject, session, path, 
                       input_type,n_channels, window_size,extend, extend_size, center, whiten, ratio, noise_db, std,
                      input_shape, dims, patch_size, n_patches,
                       n_transformer_layers, n_heads, transformer_units,
                       mlp_head_units, n_classes,stochastic_depth,stochastic_depth_rate, type_of_experiment)

************************************************
Experiment Two
************************************************
####################################################################################
Loading subject 1
####################################################################################
Whiten.
Size of the input data is (17152, 64, 102, 1)
The input label shape is (17152,)
The total number of classes is 34
************************************************************************************
Loaded RAW input data
************************************************************************************
Adding noise to RAW input test data
Adding white noise to RAW input test data
---------------------------------------------------
Training for fold 1 -------
Epoch 1/150


2023-09-04 17:35:11.252444: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8600
2023-09-04 17:35:24.436831: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


358/358 [==============================] - 35s 13ms/step - loss: 3.2357 - accuracy: 0.0847
Epoch 2/150
358/358 [==============================] - 5s 13ms/step - loss: 2.1890 - accuracy: 0.2823
Epoch 3/150
358/358 [==============================] - 5s 13ms/step - loss: 1.7260 - accuracy: 0.4117
Epoch 4/150
358/358 [==============================] - 5s 13ms/step - loss: 1.4532 - accuracy: 0.4948
Epoch 5/150
358/358 [==============================] - 5s 14ms/step - loss: 1.2666 - accuracy: 0.5620
Epoch 6/150
358/358 [==============================] - 5s 14ms/step - loss: 1.1761 - accuracy: 0.5952
Epoch 7/150
358/358 [==============================] - 5s 13ms/step - loss: 1.0888 - accuracy: 0.6267
Epoch 8/150
358/358 [==============================] - 5s 14ms/step - loss: 0.9924 - accuracy: 0.6587
Epoch 9/150
358/358 [==============================] - 5s 13ms/step - loss: 0.9366 - accuracy: 0.6838
Epoch 10/150
358/358 [==============================] - 5s 13ms/step - loss: 0.8847 - accurac

/home/bmis/anaconda3/envs/tf/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


134/134 [==============================] - 0s 3ms/step
####################################################################################
Loading subject 2
####################################################################################
Whiten.
Size of the input data is (17279, 64, 102, 1)
The input label shape is (17279,)
The total number of classes is 34
************************************************************************************
Loaded RAW input data
************************************************************************************
Adding noise to RAW input test data
Adding white noise to RAW input test data
---------------------------------------------------
Training for fold 1 -------
Epoch 1/150
360/360 [==============================] - 7s 14ms/step - loss: 3.4877 - accuracy: 0.0530
Epoch 2/150
360/360 [==============================] - 5s 13ms/step - loss: 2.8931 - accuracy: 0.1676
Epoch 3/150
360/360 [==============================] - 5s 13ms/step - loss: 2.278

...

D:\anaconda\envs\decomp\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


136/136 [==============================] - 1s 7ms/step


D:\anaconda\envs\decomp\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


136/136 [==============================] - 1s 7ms/step


PermissionError: [Errno 13] Permission denied: '../results\\Experiment_2_raw_1_to_20_whitening_.csv'